In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

from GaborExtractor import GaborExtractor
from utils import read_processed_data

In [2]:
data, labels = read_processed_data('../Preprocessed Dataset')
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.25, random_state=42)

In [3]:
# Define parameters
orientations = [k * np.pi / 8 for k in range(1, 9)]
frequencies = np.linspace(0.2, 0.5, 3)
sigmas = np.linspace(5, 1, 3)

In [4]:
# Generate and apply Gabor filters
gabor_extractor_training = GaborExtractor()
X_train = gabor_extractor_training.extract_gabor_features(X_train, orientations, frequencies, sigmas)
y_train = np.array(y_train)
X_train.shape

Prepared Filters
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
2

KeyboardInterrupt: 

In [ ]:
# Define the classifiers
classifiers = {
    "Logistic Regression": LogisticRegression(),
    "LDA": LinearDiscriminantAnalysis(),
    "QDA": QuadraticDiscriminantAnalysis(),
    "SVM": svm.SVC(),
    "Decision Tree": DecisionTreeClassifier()
}

# Train and test each classifier
for name, clf in classifiers.items():
    # Train the classifier
    clf.fit(X_train, y_train)
    
    # Predict on the training data
    y_train_pred = clf.predict(X_train)
    
    # Calculate and print the result statistics
    print(f"Classification Report on {name}:\n", classification_report(y_train, y_train_pred))

c:\Users\Abdelaal\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Classification Report on Logistic Regression:
               precision    recall  f1-score   support

           0       0.91      0.92      0.91       575
           1       0.98      0.96      0.97       605
           2       0.92      0.89      0.91       621
           3       0.90      0.94      0.92       597

    accuracy                           0.93      2398
   macro avg       0.93      0.93      0.93      2398
weighted avg       0.93      0.93      0.93      2398

Classification Report on LDA:
               precision    recall  f1-score   support

           0       0.98      0.97      0.98       575
           1       0.99      0.99      0.99       605
           2       0.97      0.97      0.97       621
           3       0.95      0.98      0.97       597

    accuracy                           0.98      2398
   macro avg       0.98      0.98      0.98      2398
weighted avg       0.98      0.98      0.98      2398

Classification Report on QDA:
               precisi

In [ ]:
# Ensure y_train is a 2D column vector
y_train = y_train.reshape(-1, 1)

# Combine the arrays
combined_array = np.hstack((X_train, y_train))

# Save the combined array to a file
np.savetxt("Saved Features/Gabor_features_training_2.csv", combined_array, delimiter=",", fmt='%.16f')

In [ ]:
# Generate and apply Gabor filters
gabor_extractor_validation = GaborExtractor()
X_val = gabor_extractor_validation.extract_gabor_features(X_val, orientations, frequencies, sigmas)
y_val = np.array(y_val)
X_val.shape

Prepared Filters
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
2

(800, 50)

In [ ]:
# Train and test each classifier
for name, clf in classifiers.items():    
    # Predict on the training data
    y_val_pred = clf.predict(X_val)
    
    # Calculate and print the result statistics
    print(f"Classification Report on {name}:\n", classification_report(y_val, y_val_pred))

Classification Report on Logistic Regression:
               precision    recall  f1-score   support

           0       0.90      0.88      0.89       225
           1       0.95      0.96      0.96       193
           2       0.90      0.91      0.90       179
           3       0.91      0.91      0.91       203

    accuracy                           0.91       800
   macro avg       0.91      0.91      0.91       800
weighted avg       0.91      0.91      0.91       800

Classification Report on LDA:
               precision    recall  f1-score   support

           0       0.97      0.97      0.97       225
           1       0.99      0.97      0.98       193
           2       0.97      0.97      0.97       179
           3       0.98      1.00      0.99       203

    accuracy                           0.98       800
   macro avg       0.98      0.98      0.98       800
weighted avg       0.98      0.98      0.98       800

Classification Report on QDA:
               precisi

In [ ]:
# Ensure y_train is a 2D column vector
y_val = y_val.reshape(-1, 1)

# Combine the arrays
combined_array = np.hstack((X_val, y_val))

# Save the combined array to a file
np.savetxt("Saved Features/Gabor_features_validation_2.csv", combined_array, delimiter=",", fmt='%.16f')

In [ ]:
# Load the data from the file, assuming no missing values but using genfromtxt for its flexibility
data = np.genfromtxt("Saved Features/Gabor_features_training_2.csv", delimiter=",")

# Splitting into features and target variable
X_train_loaded = data[:, :-1]
y_train_loaded = data[:, -1]

In [ ]:
# Load the data from the file, assuming no missing values but using genfromtxt for its flexibility
data = np.genfromtxt("Saved Features/Gabor_features_validation_2.csv", delimiter=",")

# Splitting into features and target variable
X_val_loaded = data[:, :-1]
y_val_loaded = data[:, -1]

In [ ]:
np.all(y_train.reshape(-1) == y_train_loaded)

True

In [ ]:
np.all(y_val.reshape(-1) == y_val_loaded)

True

In [ ]:
X_train[0]

array([0.40474707, 1.0635253 , 0.12097283, 0.41414946, 0.03062857,
       0.18024863, 0.51819015, 1.1385502 , 0.15061918, 0.4176489 ,
       0.11766002, 0.33240256, 0.3430412 , 0.9088282 , 0.09519599,
       0.34150752, 0.02995236, 0.17861196, 0.3265634 , 0.96023744,
       0.24053459, 0.6058825 , 0.0137053 , 0.11626463, 0.34886804,
       0.9197886 , 0.09764568, 0.34642255, 0.03222549, 0.18448313,
       0.4890861 , 1.0444945 , 0.15057422, 0.4064375 , 0.11528882,
       0.3267434 , 0.29102933, 0.76629514, 0.09324453, 0.32755122,
       0.03052907, 0.18012688, 0.32212275, 0.92758274, 0.22555564,
       0.5468712 , 0.01368588, 0.11618339, 1.1323183 , 1.8529645 ],
      dtype=float32)

In [ ]:
X_train_loaded[0]

array([0.40474707, 1.06352532, 0.12097283, 0.41414946, 0.03062857,
       0.18024863, 0.51819015, 1.13855016, 0.15061918, 0.41764891,
       0.11766002, 0.33240256, 0.34304121, 0.9088282 , 0.09519599,
       0.34150752, 0.02995236, 0.17861196, 0.32656339, 0.96023744,
       0.24053459, 0.60588253, 0.0137053 , 0.11626463, 0.34886804,
       0.9197886 , 0.09764568, 0.34642255, 0.03222549, 0.18448313,
       0.48908609, 1.04449451, 0.15057422, 0.40643749, 0.11528882,
       0.32674339, 0.29102933, 0.76629514, 0.09324453, 0.32755122,
       0.03052907, 0.18012688, 0.32212275, 0.92758274, 0.22555564,
       0.54687119, 0.01368588, 0.11618339, 1.13231826, 1.85296452])